In [34]:
import numpy as np
import pandas as pd
import json
import requests
import ast

from imdb import IMDb
ia = IMDb()

import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn')
%matplotlib inline

In [35]:
from eric_keys import eric_omdb_key, eric_tmdb_key, eric_tmdb_token

In [52]:
# Read csv
df = pd.read_csv('clean_df.csv', index_col='id').sort_index()
df

,imdb_id,budget,revenue,genre_names,original_title,runtime,status,countries,title,original_language,avg_vote,vote_count,popularity,genres,year
id,,,,,,,,,,,,,,,
55,tt0245712,2000000,20908467.0,"['Drama', 'Thriller']",Amores perros,154.0,Released,['Mexico'],Amores Perros,es,7.6,1257,16.137,"[18, 53]",2000
64,tt0287467,0,64790996.0,"['Drama', 'Romance']",Hable con ella,112.0,Released,['Spain'],Talk to Her,es,7.6,639,12.528,"[18, 10749]",2002
79,tt0299977,31000000,177395557.0,"['Drama', 'Adventure', 'Action', 'History']",英雄,99.0,Released,['China'],Hero,zh,7.5,1261,24.119,"[18, 12, 28, 36]",2002
99,tt0185125,8272296,67952969.0,"['Comedy', 'Drama']",Todo sobre mi madre,101.0,Released,"['Spain', 'France']",All About My Mother,es,7.7,932,14.084,"[35, 18]",1999
104,tt0130827,1530000,7267585.0,"['Action', 'Drama', 'Thriller']",Lola rennt,81.0,Released,['Germany'],Run Lola Run,de,7.3,1247,15.865,"[28, 18, 53]",1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637920,tt10431500,0,17321629.0,"['Drama', 'Comedy', 'Family']",7. Koğuştaki Mucize,132.0,Released,['Turkey'],Miracle in Cell No. 7,tr,8.4,1927,78.940,"[35, 18, 10751]",2019
652483,tt10731748,0,NaN,"['Comedy', 'Romance']",Modo Avião,96.0,Released,['Brazil'],Airplane Mode,pt,6.2,283,15.085,"[35, 10749]",2020
658760,tt9806322,0,NaN,"['Romance', 'Comedy']",Isi & Ossi,113.0,Released,['Germany'],Isi & Ossi,de,7.0,386,19.938,"[35, 10749]",2020


In [38]:
# Create list of top languages by num of movies
language_count = dict(df['original_language'].value_counts())
language_count = [(key, value) for key, value in language_count.items()]

In [76]:
# Create dict for top five languages by num of movies
top_five_lang = {}
for x in language_count[0:5]:
    top_five_lang[x[0]] = {'movie_count':x[1]}

In [77]:
top_five_lang

{'fr': {'movie_count': 360},
 'it': {'movie_count': 192},
 'ja': {'movie_count': 118},
 'es': {'movie_count': 91},
 'de': {'movie_count': 43}}

In [95]:
# Add mean vote count
for key, value in top_five_lang.items():
    top_five_lang[key]['mean_vote_count'] = int(round(
        df.groupby('original_language')['vote_count'].mean().loc[key]))

In [96]:
# Add median vote count
for key, value in top_five_lang.items():
    top_five_lang[key]['median_vote_count'] = int(round(
        df.groupby('original_language')['vote_count'].median().loc[key]))

In [98]:
top_five_lang

{'fr': {'movie_count': 360, 'mean_vote_count': 555, 'median_vote_count': 386},
 'it': {'movie_count': 192, 'mean_vote_count': 499, 'median_vote_count': 332},
 'ja': {'movie_count': 118, 'mean_vote_count': 689, 'median_vote_count': 348},
 'es': {'movie_count': 91, 'mean_vote_count': 659, 'median_vote_count': 406},
 'de': {'movie_count': 43, 'mean_vote_count': 596, 'median_vote_count': 386}}

In [147]:
# Create decades columns
filt90_1 = df['year'] >= 1990
filt90_2 = df['year'] < 2000
df['90s']=np.where(filt90_1 & filt90_2,1,0)

filt00_1 = df['year'] >= 2000
filt00_2 = df['year'] < 2010
df['00s']=np.where(filt00_1 & filt00_2,1,0)

filt10_1 = df['year'] >= 2010
filt10_2 = df['year'] < 2020
df['10s']=np.where(filt10_1 & filt10_2,1,0)

In [151]:
df.to_csv('clean_df.csv')